In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image

def analyze_and_correct_detections(image, detections, categories):
    """
    Analyze detections and apply corrections if needed
    """
    corrected_detections = []
    
    for box in detections.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        
        # Extract the fruit region for additional verification
        fruit_region = image[y1:y2, x1:x2]
        
        # Calculate color statistics (BGR format)
        avg_color = np.mean(fruit_region, axis=(0, 1))
        blue, green, red = avg_color

        if class_id == 1:

            if green > 100 and red > 100 and blue < 100:

                class_id = 0
                confidence = min(confidence + 0.1, 1.0)  # Boost confidence slightly
        
        corrected_detections.append((x1, y1, x2, y2, class_id, confidence))
    
    return corrected_detections

def test_yolo_fruit_classification(yolo_model_path, image_path, conf_threshold=0.5):
    # Load YOLO model
    model = YOLO(yolo_model_path)
    
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image at {image_path}")
        return
    
    # Run inference with confidence threshold
    results = model(image, conf=conf_threshold, iou=0.45)[0]
    
    # Class names
    categories = ["Healthy", "Rotten"]
    
    # Analyze and potentially correct detections
    corrected_dets = analyze_and_correct_detections(image, results, categories)
    
    # Visualize results
    for det in corrected_dets:
        x1, y1, x2, y2, class_id, confidence = det
        label = f"{categories[class_id]} ({confidence:.2f})"
        color = (0, 255, 0) if class_id == 0 else (0, 0, 255)  # Green/Red
        
        # Draw box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        print(f"Final classification: {label} at ({x1},{y1})-({x2},{y2})")
    
    # Display result
    cv2.imshow("YOLO Fruit Detection with Correction", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Save the result
    output_path = image_path.replace(".png", "_corrected.png")
    cv2.imwrite(output_path, image)
    print(f"Result saved to {output_path}")

if __name__ == "__main__":
    yolo_weights = r"C:\Users\Acer\OneDrive\Documents\final\runs\detect\train\weights\best.pt"
    test_image = r"C:\Users\Acer\OneDrive\Documents\final\test\test50.png"
    
    # Try different confidence thresholds between 0.4-0.7
    test_yolo_fruit_classification(yolo_weights, test_image, conf_threshold=0.5)


0: 640x608 1 Rotten, 219.7ms
Speed: 12.6ms preprocess, 219.7ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 608)
Final classification: Rotten (0.73) at (209,445)-(1273,1404)
Result saved to C:\Users\Acer\OneDrive\Documents\final\test\test50_corrected.png


In [6]:
import cv2
import numpy as np
from ultralytics import YOLO

def analyze_and_correct_detections(image, detections, categories):

    corrected_detections = []
    
    for box in detections.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])

        fruit_region = image[y1:y2, x1:x2]
        
        avg_color = np.mean(fruit_region, axis=(0, 1))
        blue, green, red = avg_color

        if class_id == 1: 

            if green > 100 and red > 100 and blue < 100:

                class_id = 0
                confidence = min(confidence + 0.1, 1.0)  # Boost confidence slightly
        
        corrected_detections.append((x1, y1, x2, y2, class_id, confidence))
    
    return corrected_detections

def calculate_rotten_percentage(fruit_region):
    """
    Calculate the percentage of the rotten area in the given fruit region
    """
    # Convert the fruit region to grayscale
    gray_fruit = cv2.cvtColor(fruit_region, cv2.COLOR_BGR2GRAY)

    # Apply a simple threshold to isolate the rotten part (assuming it appears darker)
    _, binary_mask = cv2.threshold(gray_fruit, 100, 255, cv2.THRESH_BINARY_INV)

    # Calculate the area of the rotten part
    rotten_area = np.sum(binary_mask == 255)
    
    # Calculate the total area of the fruit
    total_area = fruit_region.size / 3  # Each pixel has 3 channels (BGR)

    # Calculate the percentage of rotten area
    rotten_percentage = (rotten_area / total_area) * 100
    
    return rotten_percentage, binary_mask

def highlight_rotten_area(image, x1, y1, x2, y2, binary_mask):
    """
    Highlight the rotten area in the image by overlaying the binary mask
    """
    # Overlay the binary mask on the original image
    highlighted_image = image.copy()

    # Apply the mask in red (BGR format)
    highlighted_image[y1:y2, x1:x2][binary_mask == 255] = [192, 192, 192]  # Silver color in BGR

    return highlighted_image

def test_yolo_fruit_classification(yolo_model_path, image_path, conf_threshold=0.5):
    # Load YOLO model
    model = YOLO(yolo_model_path)
    
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image at {image_path}")
        return
    
    # Run inference with confidence threshold
    results = model(image, conf=conf_threshold, iou=0.45)[0]
    
    # Class names
    categories = ["Healthy", "Rotten"]
    
    # Analyze and potentially correct detections
    corrected_dets = analyze_and_correct_detections(image, results, categories)
    
    # List to store rotten percentages
    rotten_percentages = []
    
    # Visualize results
    for det in corrected_dets:
        x1, y1, x2, y2, class_id, confidence = det
        label = f"{categories[class_id]} ({confidence:.2f})"
        color = (0, 255, 0) if class_id == 0 else (0, 0, 255)  # Green for Healthy, Red for Rotten
        
        # Extract the fruit region for rotten percentage calculation
        fruit_region = image[y1:y2, x1:x2]
        
        if class_id == 1:  # If the fruit is rotten
            # Calculate the rotten percentage
            rotten_percentage, binary_mask = calculate_rotten_percentage(fruit_region)
            rotten_percentages.append(rotten_percentage)
            print(f"Rotten percentage for fruit at ({x1},{y1})-({x2},{y2}): {rotten_percentage:.2f}%")
            
            # Highlight the rotten area
            image = highlight_rotten_area(image, x1, y1, x2, y2, binary_mask)
        
        # Draw box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Display result
    cv2.imshow("YOLO Fruit Detection with Rotten Highlighting", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Save the result
    output_path = image_path.replace(".png", "_highlighted.png")
    cv2.imwrite(output_path, image)
    print(f"Result saved to {output_path}")

    # Return the rotten percentages for all detected fruits
    return rotten_percentages

if __name__ == "__main__":
    yolo_weights = r"C:\Users\Acer\OneDrive\Documents\final\runs\detect\train\weights\best.pt"
    test_image = r"downloads/download.jpg"
    
    # Try different confidence thresholds between 0.4-0.7
    rotten_percentages = test_yolo_fruit_classification(yolo_weights, test_image, conf_threshold=0.5)
    
    # Print all rotten percentages
    for i, percentage in enumerate(rotten_percentages):
        print(f"Fruit {i+1} Rotten Percentage: {percentage:.2f}%")



0: 352x640 3 Rottens, 197.6ms
Speed: 6.6ms preprocess, 197.6ms inference, 1.8ms postprocess per image at shape (1, 3, 352, 640)
Rotten percentage for fruit at (99,36)-(193,127): 1.38%
Rotten percentage for fruit at (198,30)-(294,128): 0.70%
Rotten percentage for fruit at (6,45)-(96,128): 37.51%
Result saved to downloads/download.jpg
Fruit 1 Rotten Percentage: 1.38%
Fruit 2 Rotten Percentage: 0.70%
Fruit 3 Rotten Percentage: 37.51%
